###Ingest Qualifying data from bronze to silver

1. Imports

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, current_timestamp

2. Create schema

In [0]:
qualifying_schema = StructType(
    fields=[
        StructField("qualifyId", IntegerType(), False),
        StructField("raceId", IntegerType(), False),
        StructField("driverId", IntegerType(), False),
        StructField("constructorId", IntegerType(), False),
        StructField("number", IntegerType(), False),
        StructField("position", IntegerType(), True),
        StructField("q1", StringType(), True),
        StructField("q2", StringType(), True),
        StructField("q3", StringType(), True)
    ]
)

3. Read qualifying/*.json file

In [0]:
qualifying_raw_df = spark.read \
    .format('json') \
    .option("multiline", True) \
    .schema(qualifying_schema) \
    .load("abfss://bronze@formula1faijdl.dfs.core.windows.net/qualifying/*.json")

4. Transform the circuit dataframe

In [0]:
qualifying_df = qualifying_raw_df \
        .select(
                col("qualifyId").alias("qualify_id"),
                col("raceId").alias("race_id"),
                col("driverId").alias("driver_id"),
                col("constructorId").alias("constructor_id"),
                col('number'),
                col("position"),
                col("q1"),
                col("q2"),
                col("q3"),
                current_timestamp().alias('ingestion_date')
        )

5. Write the dataframe to silver layer

In [0]:
qualifying_df.write.mode('overwrite').parquet("abfss://silver@formula1faijdl.dfs.core.windows.net/qualifying")